In [1]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv('gurgaon_properties_cleaned_v1.csv')

In [5]:
df.isnull().sum()

property_type         0
society               1
sector                0
price                18
price_per_sqft       18
area                 18
areaWithType          0
bedRoom               0
bathroom              0
balcony               0
additionalRoom        0
floorNum             19
facing             1105
agePossession         1
nearbyLocations     177
furnishDetails      981
features            635
dtype: int64

In [6]:
df.duplicated().sum()

122

In [7]:
df.head(1)
# focus is on -> areaWithType, additionalRoom, agePossession, furnishDetails, features 



,property_type,society,sector,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,floorNum,facing,agePossession,nearbyLocations,furnishDetails,features
0,flat,ramprasta awho,sector 95,0.92,5257.0,1750.0,Built Up area: 1750 (162.58 sq.m.)Carpet area:...,3,3,3+,not available,13.0,North-East,undefined,NaN,NaN,NaN


In [13]:
#areaWithType
import re

# This function extracts the Super Built up area
def get_super_built_up_area(text):
    match = re.search(r'Super Built up area (\d+\.?\d*)', text)
    if match:
        return float(match.group(1))
    return None

In [14]:
# This function extracts the Built Up area or Carpet area
def get_area(text, area_type):
    match = re.search(area_type + r'\s*:\s*(\d+\.?\d*)', text)
    if match:
        return float(match.group(1))
    return None

In [15]:


# This function checks if the area is provided in sq.m. and converts it to sqft if needed
def convert_to_sqft(text, area_value):
    if area_value is None:
        return None
    match = re.search(r'{} \((\d+\.?\d*) sq.m.\)'.format(area_value), text)
    if match:
        sq_m_value = float(match.group(1))
        return sq_m_value * 10.7639  # conversion factor from sq.m. to sqft
    return area_value



In [16]:


# Extract Super Built up area and convert to sqft if needed
df['super_built_up_area'] = df['areaWithType'].apply(get_super_built_up_area)
df['super_built_up_area'] = df.apply(lambda x: convert_to_sqft(x['areaWithType'], x['super_built_up_area']), axis=1)

# Extract Built Up area and convert to sqft if needed
df['built_up_area'] = df['areaWithType'].apply(lambda x: get_area(x, 'Built Up area'))
df['built_up_area'] = df.apply(lambda x: convert_to_sqft(x['areaWithType'], x['built_up_area']), axis=1)

# Extract Carpet area and convert to sqft if needed
df['carpet_area'] = df['areaWithType'].apply(lambda x: get_area(x, 'Carpet area'))
df['carpet_area'] = df.apply(lambda x: convert_to_sqft(x['areaWithType'], x['carpet_area']), axis=1)



In [18]:
df[['price','property_type','area','areaWithType','super_built_up_area','built_up_area','carpet_area']].sample(5)

,price,property_type,area,areaWithType,super_built_up_area,built_up_area,carpet_area
3114,2.17,flat,1550.0,Carpet area: 1549 (143.91 sq.m.),NaN,NaN,1549.0
3791,1.60,flat,2545.0,Super Built up area 2545(236.44 sq.m.),2545.0,NaN,NaN
3393,1.25,flat,1744.0,Super Built up area 1744(162.02 sq.m.),1744.0,NaN,NaN
759,1.60,flat,1950.0,Super Built up area 1950(181.16 sq.m.)Carpet a...,1950.0,NaN,1161.0
1222,0.95,flat,1385.0,Super Built up area 1385(128.67 sq.m.)Built Up...,1385.0,940.0,845.0


In [19]:
df[~((df['super_built_up_area'].isnull()) | (df['built_up_area'].isnull()) | (df['carpet_area'].isnull()))][['price','property_type','area','areaWithType','super_built_up_area','built_up_area','carpet_area']].shape

(534, 7)

In [20]:
df[df['areaWithType'].str.contains('Plot')][['price','property_type','area','areaWithType','super_built_up_area','built_up_area','carpet_area']].head(5)

,price,property_type,area,areaWithType,super_built_up_area,built_up_area,carpet_area
2,0.42,house,50.0,Plot area 50(4.65 sq.m.),NaN,NaN,NaN
3,6.50,house,1836.0,Plot area 215(179.77 sq.m.)Built Up area: 215 ...,NaN,215.0,NaN
15,8.50,house,4000.0,Plot area 4000(371.61 sq.m.),NaN,NaN,NaN
16,3.51,house,2844.0,Plot area 316(264.22 sq.m.),NaN,NaN,NaN
18,2.40,house,2000.0,Plot area 2000(185.81 sq.m.),NaN,NaN,NaN


In [21]:
df.isnull().sum()

property_type             0
society                   1
sector                    0
price                    18
price_per_sqft           18
area                     18
areaWithType              0
bedRoom                   0
bathroom                  0
balcony                   0
additionalRoom            0
floorNum                 19
facing                 1105
agePossession             1
nearbyLocations         177
furnishDetails          981
features                635
super_built_up_area    1888
built_up_area          2616
carpet_area            1859
dtype: int64

In [22]:
all_nan_df = df[((df['super_built_up_area'].isnull()) & (df['built_up_area'].isnull()) & (df['carpet_area'].isnull()))][['price','property_type','area','areaWithType','super_built_up_area','built_up_area','carpet_area']]

In [23]:
all_nan_index = df[((df['super_built_up_area'].isnull()) & (df['built_up_area'].isnull()) & (df['carpet_area'].isnull()))][['price','property_type','area','areaWithType','super_built_up_area','built_up_area','carpet_area']].index

In [24]:


# Function to extract plot area from 'areaWithType' column
def extract_plot_area(area_with_type):
    match = re.search(r'Plot area (\d+\.?\d*)', area_with_type)
    return float(match.group(1)) if match else None



In [25]:
all_nan_df['built_up_area'] = all_nan_df['areaWithType'].apply(extract_plot_area)

# Update the original dataframe
#gurgaon_properties.update(filtered_rows)

In [26]:
all_nan_df

,price,property_type,area,areaWithType,super_built_up_area,built_up_area,carpet_area
2,0.42,house,50.0,Plot area 50(4.65 sq.m.),NaN,50.0,NaN
15,8.50,house,4000.0,Plot area 4000(371.61 sq.m.),NaN,4000.0,NaN
16,3.51,house,2844.0,Plot area 316(264.22 sq.m.),NaN,316.0,NaN
18,2.40,house,2000.0,Plot area 2000(185.81 sq.m.),NaN,2000.0,NaN
29,3.00,house,558.0,Plot area 62(51.84 sq.m.),NaN,62.0,NaN
...,...,...,...,...,...,...,...
3768,2.90,house,2500.0,Plot area 2500(232.26 sq.m.),NaN,2500.0,NaN
3771,0.80,house,100.0,Plot area 100(9.29 sq.m.),NaN,100.0,NaN
3774,8.40,house,4000.0,Plot area 4000(371.61 sq.m.),NaN,4000.0,NaN
3781,4.70,house,2160.0,Plot area 240(200.67 sq.m.),NaN,240.0,NaN


In [27]:


def convert_scale(row):
    if np.isnan(row['area']) or np.isnan(row['built_up_area']):
        return row['built_up_area']
    else:
        if round(row['area']/row['built_up_area']) == 9.0:
            return row['built_up_area'] * 9
        elif round(row['area']/row['built_up_area']) == 11.0:
            return row['built_up_area'] * 10.7
        else:
            return row['built_up_area']



In [28]:


all_nan_df['built_up_area'] = all_nan_df.apply(convert_scale,axis=1)



In [29]:
all_nan_df

,price,property_type,area,areaWithType,super_built_up_area,built_up_area,carpet_area
2,0.42,house,50.0,Plot area 50(4.65 sq.m.),NaN,50.0,NaN
15,8.50,house,4000.0,Plot area 4000(371.61 sq.m.),NaN,4000.0,NaN
16,3.51,house,2844.0,Plot area 316(264.22 sq.m.),NaN,2844.0,NaN
18,2.40,house,2000.0,Plot area 2000(185.81 sq.m.),NaN,2000.0,NaN
29,3.00,house,558.0,Plot area 62(51.84 sq.m.),NaN,558.0,NaN
...,...,...,...,...,...,...,...
3768,2.90,house,2500.0,Plot area 2500(232.26 sq.m.),NaN,2500.0,NaN
3771,0.80,house,100.0,Plot area 100(9.29 sq.m.),NaN,100.0,NaN
3774,8.40,house,4000.0,Plot area 4000(371.61 sq.m.),NaN,4000.0,NaN
3781,4.70,house,2160.0,Plot area 240(200.67 sq.m.),NaN,2160.0,NaN


In [30]:
# update the original dataframe
df.update(all_nan_df)

In [31]:
df.head()

,property_type,society,sector,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,floorNum,facing,agePossession,nearbyLocations,furnishDetails,features,super_built_up_area,built_up_area,carpet_area
0,flat,ramprasta awho,sector 95,0.92,5257.0,1750.0,Built Up area: 1750 (162.58 sq.m.)Carpet area:...,3,3,3+,not available,13.0,North-East,undefined,NaN,NaN,NaN,NaN,1750.0,1600.0
1,flat,bestech park view grand spa,sector 81,2.20,8270.0,2660.0,Super Built up area 2660(247.12 sq.m.)Carpet a...,3,4,3+,servant room,8.0,East,1 to 5 Year Old,"['Sapphire 83 Mall', 'NH-8, IMT Manesar', 'Dwa...","['4 Wardrobe', '1 Water Purifier', '4 Fan', '1...","['Water purifier', 'Security / Fire Alarm', 'P...",2660.0,NaN,2300.0
2,house,ss omnia,sector 86,0.42,84000.0,50.0,Plot area 50(4.65 sq.m.),5,3,2,not available,3.0,NaN,1 to 5 Year Old,"['Orris Community Center', 'Essar Petrol Pump'...",[],NaN,NaN,50.0,NaN
3,house,independent,sector 43,6.50,35400.0,1836.0,Plot area 215(179.77 sq.m.)Built Up area: 215 ...,5,4,3+,"pooja room,servant room",3.0,South-East,5 to 10 Year Old,"['Sector 42-43 metro station', 'Huda city cent...","['10 Fan', '5 Geyser', '26 Light', '5 AC', '1 ...","['High Ceiling Height', 'False Ceiling Lightin...",NaN,215.0,NaN
4,flat,vasant appartments,sector 14,1.70,7391.0,2300.0,Carpet area: 2300 (213.68 sq.m.),5,5,2,"pooja room,study room,store room",2.0,North-West,10+ Year Old,"['Sheetla Mata Mandir', 'Hanuman Mandir', 'Hsb...","['1 Water Purifier', '10 Fan', '1 Exhaust Fan'...","['Security / Fire Alarm', 'Intercom Facility',...",NaN,NaN,2300.0


In [32]:
df.isnull().sum()

property_type             0
society                   1
sector                    0
price                    18
price_per_sqft           18
area                     18
areaWithType              0
bedRoom                   0
bathroom                  0
balcony                   0
additionalRoom            0
floorNum                 19
facing                 1105
agePossession             1
nearbyLocations         177
furnishDetails          981
features                635
super_built_up_area    1888
built_up_area          2070
carpet_area            1859
dtype: int64

In [33]:
#additionalRoom

df['additionalRoom'].value_counts()

additionalRoom
not available                                    1587
servant room                                      705
study room                                        250
others                                            225
pooja room                                        165
                                                 ... 
study room,others,servant room                      1
store room,servant room,study room,pooja room       1
servant room,store room,study room,pooja room       1
servant room,study room,others                      1
store room,study room                               1
Name: count, Length: 65, dtype: int64

In [34]:
# additional room
# List of new columns to be created
new_cols = ['study room', 'servant room', 'store room', 'pooja room', 'others']

# Populate the new columns based on the "additionalRoom" column
for col in new_cols:
    df[col] = df['additionalRoom'].str.contains(col).astype(int)

In [35]:


df.sample(5)[['additionalRoom','study room', 'servant room', 'store room', 'pooja room', 'others']]



,additionalRoom,study room,servant room,store room,pooja room,others
1721,"store room,servant room",0,1,1,0,0
1532,others,0,0,0,0,1
662,not available,0,0,0,0,0
2665,study room,1,0,0,0,0
1606,not available,0,0,0,0,0
